In [1]:
# 1) Install PDF library
!pip install reportlab -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.6 MB/s eta 0:00:00


In [67]:
# 2) Paths & loaders
import os, json, re, time, math
from typing import List, Dict, Any

BASE = "/content"
os.makedirs(f"{BASE}/reports", exist_ok=True)

# REQUIRED: mitigation outputs (from Mitigation Strategist Agent)
MITIGATION_JSON = f"{BASE}/mitigation_results.json"       # <-- change if different

STAMP = time.strftime("%Y%m%d-%H%M%S", time.gmtime())
PDF_PATH = f"{BASE}/reports/CyberMind_Report_{STAMP}.pdf"

def read_json(path: str):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            try:
                return json.load(f)
            except Exception:
                return []
    return []

mitigation_items = read_json(MITIGATION_JSON)
threat_items     = read_json(THREATS_JSON)
vuln_items       = read_json(VULN_JSON)

print("Loaded:",
      "\n- mitigation:", len(mitigation_items))


Loaded: 
- mitigation: 5


In [68]:
# Ensure mitigation_items is a list even if the file is a single dict
raw_mit = read_json(MITIGATION_JSON)
if isinstance(raw_mit, dict):
    mitigation_items = [raw_mit]
elif isinstance(raw_mit, list):
    mitigation_items = raw_mit
else:
    mitigation_items = []
print("Mitigation items (normalized to list):", len(mitigation_items))

Mitigation items (normalized to list): 1


In [69]:
# 3) Normalization helpers
def to_list(x):
    if x is None: return []
    return x if isinstance(x, list) else [x]

def short(txt, n=220):
    if not txt: return ""
    t = str(txt).replace("\n"," ").strip()
    return (t[:n]+"…") if len(t)>n else t

def parse_cvss_base(v):
    if v is None: return None
    if isinstance(v, (int,float)): return float(v)
    s = str(v)
    m = re.search(r'BASESCORE:([0-9.]+)', s, re.I)
    if m: return float(m.group(1))
    m2 = re.search(r'([0-9]\.[0-9])', s)
    return float(m2.group(1)) if m2 else None

def best_id_from_refs(refs: List[str]):
    for r in refs or []:
        if isinstance(r, str) and r.upper().startswith("CVE-"):
            return r.upper()
    return None

In [70]:
# 4) Parser for mitigation agent raw text -> dict
import re

_cve_re = re.compile(r'\bCVE-\d{4}-\d{4,7}\b', re.IGNORECASE)

def parse_mitigation_text(text: str) -> dict:
    t = text.replace("\r", "\n")

    m_sum = re.search(r'(?:-?\s*Summary\s*[:\-]\s*)(.+?)(?:\n\s*\-?\s*Mitigation|\n\s*Mitigation Steps|$)',
                      t, re.IGNORECASE | re.DOTALL)
    summary = (m_sum.group(1).strip() if m_sum else "No summary provided.")

    mitigations = []
    m_steps = re.search(r'(Mitigation Steps|Mitigations|Recommended Actions|Mitigation)\s*[:\-]\s*(.+?)(?:\n\s*\-?\s*Priority|\n\s*Priority|$)',
                        t, re.IGNORECASE | re.DOTALL)
    if m_steps:
        steps_block = m_steps.group(2).strip()
        parts = re.split(r'\n\s*\d+\.\s+|\n\s*-\s+|\n\s*\*\s+', "\n" + steps_block)
        for s in parts:
            s = s.strip()
            if len(s) > 3:
                s = re.sub(r'\bReferences?\b\s*[:\-].*$', '', s, flags=re.IGNORECASE | re.DOTALL).strip()
                if s:
                    mitigations.append(' '.join(s.split()))
    if not mitigations:
        for line in t.splitlines():
            if re.search(r'\b(patch|update|apply|enable|use|monitor|restrict|block|validate|sanitize|harden|audit)\b', line, re.IGNORECASE):
                s = line.strip().lstrip('-*0123456789. ')
                if len(s) > 6:
                    mitigations.append(s)
    mitigations = list(dict.fromkeys(mitigations))

    m_pr = re.search(r'Priority\s*[:\-]\s*(\w+)', t, re.IGNORECASE)
    priority = m_pr.group(1).title() if m_pr else "Unknown"

    m_eff = re.search(r'Estimated Effort\s*[:\-]\s*(\w+)', t, re.IGNORECASE)
    effort = m_eff.group(1).title() if m_eff else "Unknown"

    refs = [c.upper() for c in _cve_re.findall(t)]
    urls = re.findall(r'https?://[^\s,;]+', t)
    references = list(dict.fromkeys(refs + urls))

    return {
        "summary": summary,
        "mitigations": mitigations or ["Review vendor guidance."],
        "priority": priority,
        "estimated_effort": effort,
        "references": references,
        "confidence": None,
        "rationale": None,
        "cve_ids": [r for r in references if r.upper().startswith("CVE-")]
    }

In [71]:
# 5) Build unified rows (robust: handles dict or string mitigation items)
def to_list(x):
    if x is None: return []
    return x if isinstance(x, list) else [x]

def normalize_mitigation(m):
    # If it's string (unlikely now)
    if isinstance(m, str):
        m = parse_mitigation_text(m)

    # Read your exact keys first (no spaces), then fallbacks
    summary = (
        m.get("Summary")
        or m.get("summary")
        or m.get("Overview")
        or ""
    )

    steps = (
        m.get("MitigationSteps")
        or m.get("mitigations")
        or m.get("Mitigation Steps")
        or []
    )
    if not isinstance(steps, list):
        steps = [steps] if steps else []

    priority = (
        m.get("Priority")
        or m.get("priority")
        or "Unknown"
    )
    priority = str(priority).title()

    effort = (
        m.get("EstimatedEffort")
        or m.get("estimated_effort")
        or m.get("Effort")
        or "Unknown"
    )
    effort = str(effort).title()

    refs = m.get("References") or m.get("references") or []
    if not isinstance(refs, list):
        refs = [refs] if refs else []

    # Try to infer CVEs from refs
    cves = [r for r in refs if isinstance(r, str) and r.upper().startswith("CVE-")]
    return {
        "cve_ids": cves,
        "summary": summary or "No summary provided.",
        "mitigations": steps or ["Review vendor guidance."],
        "priority": priority,
        "effort": effort,
        "references": refs,
        "confidence": m.get("confidence"),
        "rationale": m.get("rationale"),
        "source_file": m.get("source_file")
    }

def pick_first(*vals):
    for v in vals:
        if v is not None and str(v).strip() != "":
            return v
    return None

unified = []
for m in mitigation_items or []:
    mit = normalize_mitigation(m)
    cves = mit["cve_ids"] or [None]

    for cid in cves:
        threat = (threat_by_cve.get(cid, []) if cid else [])
        vuln   = (vuln_by_cve.get(cid, [])   if cid else [])

        titles, descs, cvsss, cwes, affected_all = [], [], [], [], []

        for t in threat:
            titles.append(t.get("title"))
            descs.append(t.get("description"))
            cvsss.append(t.get("cvss_v3"))
            if isinstance(t.get("affected"), list):
                affected_all += t["affected"]

        for v in vuln:
            titles.append(v.get("title"))
            descs.append(v.get("description"))
            cvsss.append(v.get("cvss_v3"))
            if v.get("cwe"):
                cwes.append(v.get("cwe"))
            if isinstance(v.get("affected"), list):
                affected_all += v["affected"]

        title = pick_first(*titles)
        description = pick_first(*descs)
        cvss_v3 = pick_first(*cvsss)
        cwe = pick_first(*cwes)
        cvss_base = parse_cvss_base(cvss_v3)

        unified.append({
            "cve_id": cid or "UNKNOWN",
            "title": title,
            "description": description,
            "cvss_v3": cvss_v3,
            "cvss_base": cvss_base,
            "cwe": cwe or "N/A",
            "affected": list(dict.fromkeys(affected_all)) if affected_all else [],
            "priority": mit["priority"],
            "effort": mit["effort"],
            "mitigations": mit["mitigations"],
            "summary": mit["summary"],
            "references": mit["references"],
            "confidence": mit["confidence"],
            "rationale": mit["rationale"],
            "mit_source_file": mit["source_file"]
        })

print("Unified records:", len(unified))

Unified records: 3


In [72]:
# 6) Sort & basic stats
from collections import Counter

def score_key(r):
    if isinstance(r.get("cvss_base"), (int,float)):
        return float(r["cvss_base"])
    if isinstance(r.get("confidence"), (int,float)):
        return float(r["confidence"])
    return 0.0

unified_sorted = sorted(unified, key=score_key, reverse=True)
prio_counts = Counter([ (r.get("priority") or "Unknown") for r in unified_sorted ])
print("Priority distribution:", dict(prio_counts))

Priority distribution: {'High': 3}


In [73]:
# Quick preview after normalization
tmp = [normalize_mitigation(x) for x in mitigation_items]
print("Sample normalized item:\n", json.dumps(tmp[0], indent=2, ensure_ascii=False))

Sample normalized item:
 {
  "cve_ids": [
    "CVE-2025-59248",
    "CVE-2025-53782",
    "CVE-2025-59249"
  ],
  "summary": "Multiple high-severity vulnerabilities in Microsoft Exchange Server can lead to unauthorized access and privilege escalation.",
  "mitigations": [
    "Apply the latest security patches and updates from Microsoft for Exchange Server.",
    "Implement strict input validation mechanisms to prevent spoofing attacks.",
    "Review and enhance authentication algorithms to ensure they are correctly implemented.",
    "Enforce strong authentication methods to mitigate weak authentication risks.",
    "Conduct regular security audits and vulnerability assessments on the Exchange Server environment."
  ],
  "priority": "High",
  "effort": "Medium",
  "references": [
    "CVE-2025-59248",
    "CVE-2025-53782",
    "CVE-2025-59249"
  ],
  "confidence": null,
  "rationale": null,
  "source_file": null
}


In [74]:
# 7) Generate PDF
from reportlab.platypus import Paragraph, Table, TableStyle
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.units import inch

# Title & description for the summary table
story.append(Paragraph("CyberMind — Security Report", Heading))
story.append(Paragraph(
    "This table provides a summary of the most recent cyber threats identified by the CyberMind system, "
    "including their severity levels, CVSS scores, and key recommended mitigations.",
    Normal
))
story.append(Spacer(1, 10))

# compact wrapping style
wrap9 = ParagraphStyle("wrap9", parent=Normal, fontSize=9, leading=11, spaceAfter=0)

def compact_mits(rec, max_chars=160, max_items=3):
    mits = rec.get("mitigations") or []
    mits = [str(m).strip() for m in mits if m]
    if not mits:
        return "—"
    bullets = " • ".join(mits[:max_items])
    return (bullets[:max_chars] + "…") if len(bullets) > max_chars else bullets

# Header row with Paragraph so wrapping is consistent
top_rows = [[
    Paragraph("<b>CVE</b>", wrap9),
    Paragraph("<b>Priority</b>", wrap9),
    Paragraph("<b>CVSS</b>", wrap9),
    Paragraph("<b>Key Mitigations</b>", wrap9),
]]

for r in unified_sorted[:12]:
    top_rows.append([
        Paragraph(str(r.get("cve_id","—")), wrap9),
        Paragraph(str(r.get("priority","Unknown")), wrap9),
        Paragraph(str(r.get("cvss_v3","—")), wrap9),
        Paragraph(compact_mits(r), wrap9),
    ])

# Make the table fill the page width nicely in Portrait:
# set fixed widths for first 3 columns, and give the last column the remaining width.
col_w1 = 1.4*inch   # CVE
col_w2 = 0.9*inch   # Priority
col_w3 = 1.1*inch   # CVSS
col_w4 = max(2.8*inch, doc.width - (col_w1 + col_w2 + col_w3))  # Key Mitigations (stretch)

tbl = Table(
    top_rows,
    repeatRows=1,
    colWidths=[col_w1, col_w2, col_w3, col_w4]
)

tbl.setStyle(TableStyle([
    ("BACKGROUND", (0,0), (-1,0), colors.HexColor("#f0f0f0")),
    ("GRID", (0,0), (-1,-1), 0.3, colors.grey),
    ("VALIGN", (0,0), (-1,-1), "TOP"),
    ("FONTNAME", (0,0), (-1,0), "Helvetica-Bold"),
    ("FONTSIZE", (0,0), (-1,0), 10),
    ("LEFTPADDING", (0,0), (-1,-1), 6),
    ("RIGHTPADDING", (0,0), (-1,-1), 6),
    ("TOPPADDING", (0,0), (-1,-1), 4),
    ("BOTTOMPADDING", (0,0), (-1,-1), 4),
    ("ROWBACKGROUNDS", (0,1), (-1,-1), [colors.whitesmoke, colors.white]),
]))

story.append(Spacer(1, 6))
story.append(tbl)
story.append(Spacer(1, 12))
story.append(PageBreak())

# Detailed Findings
story.append(Paragraph("Detailed Findings", Heading))
story.append(Spacer(1, 8))

for i, r in enumerate(unified_sorted, 1):
    story.append(Paragraph(f"{i}. {r.get('cve_id','UNKNOWN')}", SubHdr))
    # Key fields
    cvss = r.get("cvss_v3","—")
    sev  = r.get("priority","Unknown")
    eff  = r.get("effort","Unknown")
    cwe  = r.get("cwe","N/A")
    story.append(Paragraph(f"<b>Priority:</b> {sev} &nbsp;&nbsp; <b>Effort:</b> {eff} &nbsp;&nbsp; <b>CVSS:</b> {cvss} &nbsp;&nbsp; <b>CWE:</b> {cwe}", Normal))
    story.append(Spacer(1, 4))

    title = r.get("title") or "No title"
    story.append(Paragraph(f"<b>Title:</b> {title}", Normal))

    desc = r.get("description") or r.get("summary") or "No description provided."
    story.append(Paragraph(f"<b>Description:</b> {desc}", Normal))

    if r.get("affected"):
        story.append(Paragraph(f"<b>Affected:</b> {', '.join(r['affected'])}", Normal))

    story.append(Spacer(1, 4))
    story.append(Paragraph("<b>Recommended Mitigations:</b>", Normal))
    mit_list = r.get("mitigations") or []
    if mit_list:
        flow = ListFlowable([ListItem(Paragraph(m, Normal), leftIndent=12) for m in mit_list], bulletType='1')
        story.append(flow)
    else:
        story.append(Paragraph("- —", Normal))

    if r.get("references"):
        story.append(Spacer(1, 4))
        story.append(Paragraph("<b>References:</b>", Normal))
        refs_flow = ListFlowable([ListItem(Paragraph(str(x), Normal), leftIndent=12) for x in r["references"][:8]],
                                 bulletType='bullet')
        story.append(refs_flow)

    if r.get("rationale"):
        story.append(Spacer(1, 4))
        story.append(Paragraph(f"<b>Rationale:</b> {r['rationale']}", Normal))

    story.append(Spacer(1, 12))

doc.build(story)
print("✅ PDF saved:", PDF_PATH)

✅ PDF saved: /content/reports/CyberMind_Report_20251106-082604.pdf


In [52]:
# 8) Optional: HTML + CSV exports
import pandas as pd
import markdown

html_path = f"{BASE}/reports/CyberMind_Report_{STAMP}.html"
csv_path  = f"{BASE}/reports/CyberMind_Report_{STAMP}.csv"

# HTML summary
def md_escape(s):
    return str(s).replace("|","\\|")

lines = []
lines.append(f"# CyberMind — Consolidated Security Report\n")
lines.append(f"**Generated:** {time.strftime('%Y-%m-%d %H:%M:%SZ', time.gmtime())}\n")
lines.append("## Top Table")
lines.append("| CVE | Priority | CVSS | Key Mitigations |")
lines.append("|---|---|---|---|")
for r in unified_sorted[:12]:
    mit_short = "; ".join(r.get("mitigations") or [])[:120] if r.get("mitigations") else "—"
    lines.append(f"| {md_escape(r.get('cve_id','—'))} | {md_escape(r.get('priority','Unknown'))} | {md_escape(r.get('cvss_v3','—'))} | {md_escape(mit_short)} |")

html = markdown.markdown("\n".join(lines), extensions=["tables"])
with open(html_path, "w", encoding="utf-8") as f:
    f.write(f"<!doctype html><meta charset='utf-8'><style>table{{border-collapse:collapse}}td,th{{border:1px solid #ddd;padding:6px}}</style>{html}")
print("Saved HTML:", html_path)

# CSV compact
rows_csv = []
for r in unified_sorted:
    rows_csv.append({
        "cve_id": r.get("cve_id"),
        "priority": r.get("priority"),
        "effort": r.get("effort"),
        "cvss_v3": r.get("cvss_v3"),
        "title": r.get("title"),
        "summary": short(r.get("summary") or r.get("description") or "", 180),
        "mitigations": "; ".join(r.get("mitigations") or [])[:200],
        "references": ", ".join(r.get("references") or [])[:200]
    })
pd.DataFrame(rows_csv).to_csv(csv_path, index=False, encoding="utf-8")
print("Saved CSV:", csv_path)

Saved HTML: /content/reports/CyberMind_Report_20251106-072036.html
Saved CSV: /content/reports/CyberMind_Report_20251106-072036.csv
